In [20]:
####Goal: Build a tennis player comparison app

####why?
# fun sport, fun project !

####highlights
#3 different data sources
#uses pandas, tkinter and random
# creates a custom window with drop down menus,
# buttons and custom table

####pain points
# cleaning and scraping the data 
# summing up wins by player over 21 years of ATP matches
# instead of vlookup use df.merge()!
####new tools
# installing dash :/ -> so I found tkinter
# tkinter is a new interface! buttons, drop down menus

####next steps
# add photos for each player via wiki API
# so much match data

In [ ]:
import pandas as pd

#making sure that the column names display fully
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

df_players = pd.read_csv('tennis_atp-master/atp_players.csv')

#add a header to the ranking file
df_ranking = pd.read_csv('tennis_atp-master/atp_rankings_current.csv', header = None)

#OS module to grab names of files in tennis_atp-master
import os
files = os.listdir('tennis_atp-master')
files.sort()

matches = []
for f in files:
    if 'atp_matches' in f:
        matches.append(pd.read_csv(f'tennis_atp-master/{f}'))

df_temp = pd.concat(matches)

# clean matches (reduce the number of columns)
cmatches = df_temp[['tourney_id','tourney_date','tourney_name','match_num','winner_id','surface','winner_name','winner_hand','winner_ht','loser_id','loser_name','loser_hand','loser_ht','score', 'best_of', 'round','minutes', 'w_ace','l_ace']]
cmatches.to_csv('tennis_atp-master/matches_clean.csv')

In [2]:
#leaving this in as a reminder of iteration
#only run 1 time to add a header
#df_players.to_csv('tennis_atp-master/atp_players.csv', header =['player_id','first_name','last_name','hand','birth_date','country'], index = False)
#df_players = pd.read_csv('tennis_atp-master/atp_players.csv')

#run the code below only one time to add a header
#df_ranking.to_csv('tennis_atp-master/atp_rankings_current.csv', header =['date','rank','player_id','points'], index = False)
#df_ranking = pd.read_csv('tennis_atp-master/atp_rankings_current.csv')

#files = files.remove('.DS_Store')
#print(matches)

#type(cmatches)
#cmatches.head()

In [6]:
#sum up wins and loses by player over all 21 years
#save all wins by player
win_grp = cmatches.groupby('winner_id', sort=True).size()
win_grp
#df_players['wins']= win_grp
df_players.head(100)
win_grp.to_csv('tennis_atp-master/win_grp.csv')

#save all loses by player
lose_grp = cmatches.groupby('loser_id', sort = True).size()
lose_grp.to_csv('tennis_atp-master/lose_grp.csv')

win_grp = pd.DataFrame(win_grp)
win_grp.columns = ["wins"]
win_grp

lose_grp = pd.DataFrame(lose_grp)
lose_grp.columns = ["loses"]

df_tennis = pd.merge(win_grp, lose_grp, left_index = True, right_index = True)
df_tennis

,wins,loses
100644,257,130
100754,2,4
101086,7,19
101150,34,46
101185,3,7
...,...,...
208055,1,2
208518,1,1
209080,1,1
209226,2,1


In [9]:
#number of finals attended and titles won by 
finals = cmatches[(cmatches['round'] == 'F')]
    
finals_grp = finals.groupby(['winner_id','loser_id']).size().reset_index(name='finals_attended')
finals_grp
titles_grp = finals.groupby('winner_id').size()
type(titles_grp)

titles_grp = pd.DataFrame(titles_grp)
titles_grp.columns = ["titles"]

In [10]:
#count player id in wins and loses
df_finals_grp = finals_grp[['winner_id', 'loser_id']].apply(pd.Series.value_counts, result_type = 'expand')

# results in NaNs
df_finals_grp = df_finals_grp.fillna(0)
df_finals_grp

#create the attended column
attended = df_finals_grp['winner_id'] + df_finals_grp['loser_id']

df_finals_grp['attended'] = attended
#df_finals_grp.columns =('player_id','winner_id', 'loser_id','attended')
df_finals_grp.to_csv('tennis_atp-master/finals_attended.csv')
df_finals_grp

df_finals_grp = pd.DataFrame(df_finals_grp.attended)
df_finals_grp.columns = ["finals attended"]

#merging finals and titles columns
df_tennis = pd.merge(df_tennis, df_finals_grp, left_index = True, right_index = True)
df_tennis = pd.merge(df_tennis, titles_grp, left_index = True, right_index = True)

,wins,loses,finals attended,titles
100644,257,130,20.0,14
101320,36,32,1.0,1
101532,65,83,3.0,1
101611,59,58,2.0,2
101723,32,26,2.0,1
101736,272,86,22.0,16
101820,61,78,2.0,2
101885,115,144,2.0,1
101948,104,47,8.0,3
101962,171,128,10.0,4


In [11]:
df_players['name'] = df_players['first_name'] + " " + df_players['last_name']
df_players.index = df_players.player_id
df_players = df_players [['name','country','hand']]

df_tennis = pd.merge(df_tennis, df_players, left_index = True, right_index = True)


df_tennis = df_tennis[['name','country','hand','wins','loses','finals attended','titles']]
df_tennis

,name,country,hand,wins,loses,finals attended,titles
100644,Alexander Zverev,GER,R,257,130,20.0,14
101320,Magnus Gustafsson,SWE,R,36,32,1.0,1
101532,Francisco Clavet,ESP,L,65,83,3.0,1
101611,Cedric Pioline,FRA,R,59,58,2.0,2
101723,Magnus Larsson,SWE,R,32,26,2.0,1
101736,Andre Agassi,USA,R,272,86,22.0,16
101820,Marc Rosset,SUI,R,61,78,2.0,2
101885,Wayne Arthurs,AUS,L,115,144,2.0,1
101948,Pete Sampras,USA,R,104,47,8.0,3
101962,Younes El Aynaoui,MAR,R,171,128,10.0,4


In [12]:
df_tennis.to_csv('Tennis_ranking2.csv')

In [ ]:
# done 
## open

# build df that goes through matches and count wins by player
#GOAL: compare skills and attributes by player
## use flask with heroku as interface

# first_name, last_name, birth_date, country_code
# ranking, ranking_points
# hand, height (cm), wins, losses
# number of titles won
# number of finals attended
## how many aces 
## average game length (when won?)
## wins per surface (clay, grass, hard) -> win-hard, lose-hard, win-grass
## add photos from the wiki API
## add performance Graphs

#Based on a work at https://github.com/JeffSackmann.